<a href="https://colab.research.google.com/github/ChrisNiekler/NASDAQ-Dataset/blob/master/bert/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "transformers" muss installiert werden bevor man es nutzten kann.

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.1 MB/s 
     |████████████████████████████████| 636 kB 32.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Notwendige Libraries importieren

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

# BERT sollte auf einer GPU laufen, da es auf einer CPU deutlich mehr Zeit benötigt.

In [ ]:
# to check the GPU
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


## Lade das vortrainierte BERT Model und Tokenizer

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Laden der reinen Daten
BERT funktioniert ohne Tagging


In [ ]:
#but the tweets.csv path
data = pd.read_csv('/content/Tweets.csv')
data.head(5)

,Unnamed: 0,Tweets,Sentiment
0,0,Not surprised to see that #florists are mentio...,2.0
1,1,#Tesla Tint Complete! New vs Old @Tesla @elonm...,1.0
2,2,"Hai Phong #dog and #cat market, Thượng Thanh, ...",2.0
3,3,The mod list continues to grow... $tsla #tsla ...,1.0
4,4,5 Trades. 3 Wins. 2 Losses. P/L: +$891 1) $C...,1.0


# Prüfen ob sich Null-Werte in den Daten befinden

In [ ]:
(data['Tweets'].isnull()).sum()

0

In [ ]:
(data['Sentiment'].isnull()).sum()

0

# Null-Werte durch Neutrale Werte ersetzten
(Löschen wäre eine Alternative)

In [ ]:
data = data.fillna(value=1.0)

In [ ]:
(data['Tweets'].isnull()).sum()

0

In [ ]:
(data['Sentiment'].isnull()).sum()

0

# Aufteilen der Daten in Feature und Label

In [ ]:
x = data['Tweets']
y = data['Sentiment']

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

In [ ]:
test_list = test_x.to_list()

In [ ]:
test_list_2 = test_y.to_list()

In [ ]:
# assign data of lists.
train_da = {'Tweet': train_x, 'Sentiment': train_y}
# Create DataFrame.
train_data = pd.DataFrame(train_da)
# Print the output.
print(train_data)

                                                   Tweet  Sentiment
33884  5 burning questions the Raiders must answer in...        0.0
37757  Stewart Butterfield says Microsoft sees Slack ...        0.0
11960  People who sell happiness, don't just be happy...        1.0
20382  With dis-commoded features and px pressure (in...        1.0
29992  •@ unitefortheorangutan When @ network_for_ani...        2.0
...                                                  ...        ...
17283  #NZDUSD Both targets done❤️👍🏻  For more Signal...        1.0
26079  Positivity With Practicality Is Wonderful Form...        1.0
58109  You have T~D~S.......  #SAD!  https://twitter....        2.0
2128   $NVDA runs into sellers again around 176.95:  ...        0.0
36818  #monday #positive #thoughtoftheday Today we ce...        1.0

[49228 rows x 2 columns]


In [ ]:
# assign data of lists.
test_data_dict = {'Tweet': test_x, 'Sentiment': test_y}
# Create DataFrame.
test_data = pd.DataFrame(test_data_dict)
# Print the output.
print(test_data)

                                                   Tweet  Sentiment
27829  Storytelling Over Beats Is My Passion....A Lot...        1.0
11670  More from the latest @tesla OS Early Access re...        1.0
11106  Reverse engineering the Win10 1803-to-1903 for...        2.0
57177  Whatever #HunterBiden did or didn’t do is on h...        2.0
30764  Very proud of you Mr President! Drain The Swam...        1.0
...                                                  ...        ...
58232  Trump is afraid no one wants him for a 2nd ter...        2.0
1187   $ADBE slides to lowest close since August 5th:...        0.0
53469  #Bad charactered and selfish people who always...        2.0
55923  You said those words i can't believe that you ...        2.0
46043  Re-reading "The Power of When" by @thesleepdoc...        1.0

[12307 rows x 2 columns]
